# EnigmaNet: A Neural Network Framework for Eplipesy Classification
### This is a script that uses Keras to predict epilepsy outcomes

## Script Details
This script is designed to run with the following configuration:
* Keras
* TensorFlow backend
* PlaidML backend (if using Radeon GPU)
* Tabular data (Excel or CSVs)

## Dependencies
Run the following command to pip install all dependencies
```
pip install numpy pandas sklearn matplotlib scipy keras_tqdm imblearn
```

Conda users can search for same packages and install them via `conda install <package_name>`

Also install NeuroCombat at https://github.com/ncullen93/neuroCombat

In [ ]:
# Local modules
import enigma as en
from enigmalibrary import enigmamodels as em

# Standard modules
import numpy as np

# Sklearn modules
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Keras modules
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

## Create Enigma Data Object
The Enigma data object holds the data, and enables the application of various transformations.

In [ ]:
inData = en.enigmanet(path='/Users/sid/Documents/Projects/Enigma-ML/Dataset/T1/all.csv',
                 classcol='Dx',
                 sitecol='Site',
                 dbegin='ICV',
                 dend='R_insula_surfavg',
                 fillmissing=True,
                 harmonize=True,
                 crange=['Site', 'Sex'],
                 batch='Site',
                 discrete=['Dx', 'Sex'],
                 continuous=['Age'],
                 scale=True,
                 testsplit=0.30)

In [ ]:
inData.transformdata()

In [ ]:
x_train, x_test, y_train, y_test = inData.splitdata(oversample=True)
print('Dataset containing ' + str(len(inData.dframe.loc[:, inData.classcol])) + ' observations split into:')
print('    Training: ' + str(len(y_train)))
print('    Testing: ' + str(len(y_test)))

## Create GridSearch Dict


In [ ]:
hyperparameters = {'input_dim': [x_train.shape[1]],
                   'optimizer': ['Adam', 'Nadam', 'SGD', 'RMSprop'],
                   'activation': ['relu', 'sigmoid', 'linear', 'tanh', 'softmax'],
                   'initmode': ['glorot_uniform', 'glorot_normal', 'lecun_uniform', 'uniform'],
                   'dropout': [0.1, 0.2, 0.3],
                   'units_L1': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 500],
                   'units_L2': [2, 5, 10, 15, 20, 50, 100],
                   'L2_reg_penalty': [0.01, 0.015, 0.1, 0.15, 0.2],
                   'epochs': [50, 75, 100, 150, 200, 300, 500, 1000, 2000],
                   'batch_size': [10, 20, 40, 60, 80, 100]}

enigmaClassifier = KerasClassifier(em.CreateSequentialModel,
                                   verbose=1)

grid = GridSearchCV(estimator=enigmaClassifier,
                    param_grid=hyperparameters)
grid_result = grid.fit(x_train, y_train)


print('The parameters of the best model are: ')
print(grid_result.best_params_)

## Construct Pipeline